In [1]:
import os
import string
import torch
import torch.backends.cudnn as cudnn
import torch.utils.data
import torch.nn.functional as F

from lincenseplateocr.utils import CTCLabelConverter, AttnLabelConverter
from lincenseplateocr.dataset import RawDataset, AlignCollate
from lincenseplateocr.model import Model

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def demo(opt):
    """ model configuration """
    print("Initializing converter...")  # 디버깅 추가
    if 'CTC' in opt.Prediction:
        converter = CTCLabelConverter(opt.character)
    else:
        converter = AttnLabelConverter(opt.character)
    opt.num_class = len(converter.character)
    print(f"Number of classes: {opt.num_class}")

    print("Setting input channels...")  # 디버깅 추가
    if opt.rgb:
        opt.input_channel = 3

    print("Initializing model...")  # 디버깅 추가
    model = Model(opt)
    print("Model initialized.")  # 모델이 정상적으로 초기화되었는지 확인

    # 데이터 병렬처리 설정 (문제가 발생할 수 있으므로 테스트할 필요가 있음)
    print("Setting model to DataParallel...")
    model = torch.nn.DataParallel(model).to(device)
    print("Model set to DataParallel.")

    # 모델 가중치 로드
    print(f"Loading pretrained model from {opt.saved_model}...")
    model.load_state_dict(torch.load(opt.saved_model, map_location=device))
    print("Model loaded.")

    # prepare data
    AlignCollate_demo = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=opt.PAD)
    demo_data = RawDataset(root=opt.image_folder, opt=opt)  # use RawDataset
    demo_loader = torch.utils.data.DataLoader(
        demo_data, batch_size=opt.batch_size,
        shuffle=False,
        num_workers=int(opt.workers),
        collate_fn=AlignCollate_demo, pin_memory=True)

    # predict
    model.eval()
    output_file_paths = []
    with torch.no_grad():
        for image_tensors, image_path_list in demo_loader:
            batch_size = image_tensors.size(0)
            image = image_tensors.to(device)

            length_for_pred = torch.IntTensor([opt.batch_max_length] * batch_size).to(device)
            text_for_pred = torch.LongTensor(batch_size, opt.batch_max_length + 1).fill_(0).to(device)

            if 'CTC' in opt.Prediction:
                preds = model(image, text_for_pred)
                preds_size = torch.IntTensor([preds.size(1)] * batch_size)
                _, preds_index = preds.max(2)
                preds_str = converter.decode(preds_index, preds_size)
            else:
                preds = model(image, text_for_pred, is_train=False)
                _, preds_index = preds.max(2)
                preds_str = converter.decode(preds_index, length_for_pred)

            dashed_line = '-' * 80
            head = f'{"image_path":25s}\t{"predicted_labels":25s}'
            
            print(f'{dashed_line}\n{head}\n')

            preds_prob = F.softmax(preds, dim=2)
            preds_max_prob, _ = preds_prob.max(dim=2)
            for img_name, pred, pred_max_prob in zip(image_path_list, preds_str, preds_max_prob):
                if 'Attn' in opt.Prediction:
                    pred_EOS = pred.find('[s]')
                    pred = pred[:pred_EOS]
                    pred_max_prob = pred_max_prob[:pred_EOS]

                confidence_score = pred_max_prob.cumprod(dim=0)[-1]
                
                # os.path.basename으로 파일 이름만 추출하여 output 폴더에 저장
                file_name = os.path.basename(img_name)
                
                # output 디렉토리가 없으면 생성
                if not os.path.exists(opt.output_folder):
                    os.makedirs(opt.output_folder)

                # output 폴더에 파일 작성
                log_path = os.path.join(opt.output_folder, f'{file_name}.txt')
                with open(log_path, 'a') as log:
                    log.write(f'{head}\n{dashed_line}\n')
                    print(f'{img_name:25s}\t{pred:25s}\n')
                    log.write(f'{img_name:25s}\t{pred:25s}\n')

                output_file_paths.append(log_path)

    return output_file_paths

In [4]:
# Jupyter 환경에서는 명령줄 인자를 수동으로 설정합니다.
class Opt:
    def __init__(self, image_folder, output_folder):
        self.image_folder = image_folder  # 외부에서 입력받은 이미지 폴더 경로
        self.output_folder = output_folder  # 외부에서 입력받은 출력 폴더 경로
        self.workers = 0
        self.batch_size = 32
        self.saved_model = 'lincenseplateocr/pretrained/Fine-Tuned.pth'
        self.batch_max_length = 16
        self.imgH = 32
        self.imgW = 100
        self.rgb = False
        self.character = '0123456789().JNRW_abcdef가강개걍거겅겨견결경계고과관광굥구금기김깅나남너노논누니다대댜더뎡도동두등디라러로루룰리마머명모무문므미바배뱌버베보부북비사산서성세셔소송수시아악안양어여연영오올용우울원육으을이익인자작저전제조종주중지차처천초추출충층카콜타파평포하허호홀후히ㅣ'
        self.sensitive = False
        self.PAD = False
        self.Transformation = 'TPS'
        self.FeatureExtraction = 'ResNet'
        self.SequenceModeling = 'BiLSTM'
        self.Prediction = 'Attn'
        self.num_fiducial = 20
        self.input_channel = 1
        self.output_channel = 512
        self.hidden_size = 256

In [5]:
# 직접 설정한 옵션 객체 생성
def process_images(image_folder, output_folder):
    opt = Opt(image_folder, output_folder)

    if opt.sensitive:
        opt.character = string.printable[:-6]

    cudnn.benchmark = True
    cudnn.deterministic = True
    opt.num_gpu = torch.cuda.device_count()

    # 모델 실행 및 결과 텍스트 파일 경로 리턴
    output_file_paths = demo(opt)
    return output_file_paths


In [7]:
# 예시: 외부에서 파일 경로를 받아 처리
image_folder = 'path_to_image_folder'  # 실제 이미지 폴더 경로
output_folder = 'path_to_output_folder'  # 실제 출력 폴더 경로
result_file_paths = process_images(image_folder, output_folder)
print(f"Result files saved at: {result_file_paths}")

Initializing converter...
Number of classes: 148
Setting input channels...
Initializing model...
Initializing TPS Transformation
TPS 초기화: F=20, I_size=(32, 100), I_r_size=(32, 100), I_channel_num=1
Localization Network 초기화 중...
Localization Network 초기화: F=20, I_channel_num=1
Localization Network Conv layers 초기화 중...
Localization Network Conv layers 초기화 완료
Localization Network Fully connected layers 초기화 중...
Localization Network Fully connected layers 초기화 완료
Localization Network 초기화 완료
Grid Generator 초기화 중...
Grid Generator 초기화 완료
TPS Transformation initialized
Initializing Feature Extraction: ResNet
Feature Extraction initialized with output size: 512
Initializing Sequence Modeling with BiLSTM
Sequence Modeling initialized
Initializing Prediction: Attn
Prediction initialized
Model initialized.
Setting model to DataParallel...
Model set to DataParallel.
Loading pretrained model from lincenseplateocr/pretrained/Fine-Tuned.pth...


C:\Users\darkh\anaconda3\envs\torch-book\Lib\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
C:\Devtools\project\lincenseplateocr\modules\transformation.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("inv_delta_C", torch.tensor(self._build_inv_delta_C(self.F, self.C)).float())  # F+3 x F+3
C:\Devtools\project\lincenseplateocr\modules\transformation.py:108: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather

Model loaded.
입력 이미지 크기: torch.Size([30, 1, 32, 100])
Localization Network 실행 중...
예측된 fiducial points 크기: torch.Size([30, 20, 2])
Grid 생성 중...
생성된 그리드 크기: torch.Size([30, 32, 100, 2])
grid_sample 이후 출력 크기: torch.Size([30, 1, 32, 100])
--------------------------------------------------------------------------------
image_path               	predicted_labels         

lincenseplateocr/input\1296 (4)_crop_0.jpg	경기82사1256                

lincenseplateocr/input\1296 (4)_crop_0_out1.jpg	인천84사1296                

lincenseplateocr/input\1379 (4)_crop_0.jpg	8581339                  

lincenseplateocr/input\1379 (4)_crop_0_out1.jpg	8831848                  

lincenseplateocr/input\1379 (5)_crop_0.jpg	서울81자3379                

lincenseplateocr/input\1379 (5)_crop_0_out1.jpg	서울81바1379                

lincenseplateocr/input\1459 (22)_crop_0.jpg	인천81배1459                

lincenseplateocr/input\1459 (22)_crop_0_out1.jpg	충북91아1459                

lincenseplateocr/input\1459 (43)_crop_0.jpg	8북84